In [ ]:
# ! pip install python-docx
from docx import Document
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# download the documents from https://tbinternet.ohchr.org/_layouts/15/treatybodyexternal/Download.aspx?symbolno=CERD%2FC%2FURY%2FCO%2F21-23&Lang=en

# document = Document('G1700516.docx') # spanish
# document = Document('G1700517.docx') # arabic - parsing error
# document = Document('G1700518.docx') # chinese - parsing error
document = Document('G1700519.docx') # englisch
# document = Document('G1700520.docx') # french - parsing error
# document = Document('G1700521.docx') # russian - parsing error
sections = document.sections
len(sections)

In [ ]:
paragraphs = document.paragraphs
len(paragraphs)

In [ ]:
paragraphs[0].style.name

In [ ]:
df = pd.DataFrame()
df["styles"] = [p.style.name for p in paragraphs]
df["texts"] = [p.text for p in paragraphs]
df.head()

In [ ]:
df.styles.value_counts()

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(df.styles.to_numpy().reshape(-1, 1))
enc.categories_

In [ ]:
tmp = pd.DataFrame(enc.transform(df.styles.to_numpy().reshape(-1, 1)).toarray().astype(int))
tmp = pd.merge(df, tmp, left_index=True, right_index=True)
# columns=enc.categories_
df = tmp.rename(columns={i: enc.categories_[0][i] for i in range(len(enc.categories_[0]))})
df.head()

In [ ]:
topheaders = []
superheaders = []
headers = []
subheaders = []
texts = []

topheader = np.nan
superheader = np.nan
header = np.nan
subheader = np.nan
text = ""

for i in range(len(df)-1):
    if df["Normal"][i]==1:
        topheader = df["texts"][i].lstrip('\t')
    elif df["_ H _Ch_G"][i]==1:
        superheader = df["texts"][i].lstrip('\t')
    elif df["_ H_1_G"][i]==1:
        header = df["texts"][i].lstrip('\t')
    elif df["_ H_2/3_G"][i]==1:
        subheader = df["texts"][i].lstrip('\t')
    elif df["_ Single Txt_G"][i]==1:
        tmp = df["texts"][i].lstrip('\t')
        if tmp.startswith("("):
            text += tmp
        else:
            topheaders.append(topheader)
            superheaders.append(superheader)
            headers.append(header)
            subheaders.append(subheader)
            texts.append(text)
            text = tmp
    else:
        print(i)

In [ ]:
df = pd.DataFrame([topheaders, superheaders, headers, subheaders, texts]).T
df.head()

In [ ]:
columns = ["topheader", "superheader", "header", "subheader", "text"]
df = df.rename(columns={i: columns[i] for i in range(len(enc.categories_[0]))})
df.head()

In [ ]:
df.to_csv("parsed_docx.csv", sep="#")